In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import distributions as dist

# General Setup

We have a target distribution and we will try to sample from it using Metropolis-Hastings MCMC with MALA proposal technqiues using various proposal distributions.

Some useful links were found here:  
https://en.wikipedia.org/wiki/Metropolis-adjusted_Langevin_algorithm  
http://www.mcmchandbook.net/HandbookChapter1.pdf  
https://theclevermachine.wordpress.com/2012/11/19/a-gentle-introduction-to-markov-chain-monte-carlo-mcmc/  

In [ ]:
mean1 = [2, 0]
cov1 = [[1, 0], [0, 1]]

mean2 = [-2, 0]
cov2 = [[1, 0], [0, 1]]

rv = dist.bimodal_dist(mean1, mean2, cov1, cov2)

# Samples
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(*rv.rvs(1000).T, 'o')
ax1.axis('equal')
plt.show()

x, y = np.mgrid[-10:10:.1,
                -10:10:.1]
pos = np.dstack((x, y))

# pdf
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, rv.pdf(pos))
plt.show()

# logpdf
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
ax3.contourf(x, y, rv.logpdf(pos))
plt.show()

In [ ]:
# magnitude of gradpdf
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, np.linalg.norm(rv.grad_pdf(pos), axis=0))
plt.show()

# magnitude of grad log pdf
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, np.linalg.norm(rv.grad_logpdf(pos), axis=0))
plt.show()

In [ ]:
# Acceptance rate at every step
def acceptance_rate_per_step(states):
    states = np.asarray(states)
    steps = 0
    accepted = 0
    acc_rate = []
    for i in range(1,len(states)):
        steps += 1
        if (np.isclose(states[i,0], states[i-1,0], rtol=1e-10, atol=1e-15) and 
            np.isclose(states[i,1], states[i-1,1], rtol=1e-10, atol=1e-15)):
            accepted += 1
        acc_rate.append(accepted/steps)
    return acc_rate

## MALA Proposal

We propose by suggesting the state plus the gradient of the target distribution at the state plus a small random motion.

Note, for reference, the derivate with respect to the state of our target distribution is given by:
\begin{equation}
    \frac{\partial \pi(\mathbf{x})}{\partial\mathbf{x}}=-\frac{1}{\sqrt{det(2\pi\mathbf{\Sigma})}}exp\left[-\frac{1}{2}(\mathbf{x}-\mathbf{m})^{T}\mathbf{\Sigma}^{-1}(\mathbf{x}-\mathbf{m})\right]\mathbf{\Sigma}^{-1}(\mathbf{x}-\mathbf{m})
\end{equation}

However, this is a bit irrelevant since I am looking for the gradient at the state. I.e. $\nabla\pi(\textbf{x})$ I will do it numerically.

In [ ]:
def conditional_prob(from_state, to_state, distribution, tau=1):
    to_state = np.asarray(to_state)
    from_state = np.asarray(from_state)
    return np.exp((-4 * tau)**(-1) * 
                  np.linalg.norm(to_state - from_state - tau * 
                                 distribution.grad_logpdf(from_state))**2)


def mala_step(state, distribution, tau=1):
    state = np.asarray(state)
    proposed_state = (state + 
                      tau * distribution.grad_logpdf(state) + 
                      np.sqrt(2 * tau) * scipy.stats.multivariate_normal([0,0], np.eye(2)).rvs(1))
    
    alpha = (distribution.pdf(proposed_state) * conditional_prob(proposed_state, state, distribution)/
             (distribution.pdf(state) * conditional_prob(state, proposed_state, distribution)))
    acceptance_prob = min([1, alpha])
    
    if np.random.rand() < acceptance_prob:
        return proposed_state
    else:
        return state

In [ ]:
# MCMC Execution
states = [np.array([0,0])]
steps = 5000

for i in range(steps):
    states.append(mala_step(states[-1], rv))
#     states.append(metropolis_step(states[-1], proposal_mala, rv.logpdf, rv.grad_logpdf, cond_prob_mala))
    
states = np.asarray(states)

In [ ]:
# Acceptance rate plot
acps = acceptance_rate_per_step(states)
plt.figure(figsize=(8, 4))
plt.plot(range(steps), acps)
plt.xlabel('Steps')
plt.ylabel('Acceptance Rate')
plt.show()

In [ ]:
# Sample and trace plots
df = pd.DataFrame(states, columns=["x", "y"])

plt.figure(1, figsize=(8,8))
plt.subplot(121)
plt.scatter(df.x, df.y, c=rv.logpdf(states))
plt.axis('equal')
plt.title('Samples plot')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(222)
plt.plot(np.arange(states.shape[0]), df.x)
plt.title('Trace plot')
plt.xlabel('Steps')
plt.ylabel('x')

plt.subplot(224)
plt.plot(np.arange(states.shape[0]), df.y)
plt.xlabel('Steps')
plt.ylabel('y')

plt.show()